<div style="font-weight: bold; color:#5D8AA8" align="center">
    <div style="font-size: xx-large">Procesamiento del Lenguaje Natural 2021-22</div><br>
    <div style="font-size: x-large; color:gray">Aspect opinion extraction</div><br>
    <div style="font-size: large">María Barroso - Gloria del Valle</div><br></div><hr>
</div>

In [1]:
import json
from nltk.corpus import wordnet as wn
import numpy as np
import pandas as pd

## Assignment 1: Review datasets

**yelp_hotels.json**: json containing 5,034 reviews generated by 4,148 Yelp users about 284 hotels.

**yelp_beauty_spas.json** and **yelp_restaurants.json**: which contain Yelp reviews about beauty/spa resorts and restaurants.

Each review (JSON record) has the following fields:
* *reviewerID*: the identifier of the user who wrote the review
* *asin*: the identifier of the reviewed hotel
* *reviewText*: the text of the user’s review about the hotel
* *overall*: the 1-5 Likert scale rating assigned by the user to the hotel

### Task 1.1
Loading all the hotel reviews from the Yelp hotel reviews file.

In [3]:
def load_all_json_yelp(data_name, data_path = 'yelp_dataset'):
    with open(f'{data_path}/{data_name}.json', encoding='utf-8') as f:
        reviews = json.load(f)
    numReviews = len(reviews)
    print(f'{data_name}: {numReviews} reviews loaded')
    return reviews

reviews_hotels = load_all_json_yelp('yelp_hotels')

yelp_hotels: 5034 reviews loaded


In [4]:
print(reviews_hotels[0])
print(reviews_hotels[0].get('reviewerID'))

{'reviewerID': 'qLCpuCWCyPb4G2vN-WZz-Q', 'asin': '8ZwO9VuLDWJOXmtAdc7LXQ', 'summary': 'summary', 'reviewText': "Great hotel in Central Phoenix for a stay-cation, but not necessarily a place to stay out of town and without a car. Not much around the area, and unless you're familiar with downtown, I would rather have a guest stay in Old Town Scottsdale, etc. BUT if you do stay here, it's awesome. Great boutique rooms. Awesome pool that's happening in the summer. A GREAT rooftop patio bar, and a very very busy lobby with Gallo Blanco attached. A great place to stay, but have a car!", 'overall': 4.0}
qLCpuCWCyPb4G2vN-WZz-Q


### Task 1.2
Loading line by line the reviews from the Yelp beauty/spa resorts and restaurants reviews files

In [5]:
def load_by_line_json_yelp(data_name, data_path = 'yelp_dataset'):
    reviews = []
    with open(f'{data_path}/{data_name}.json', encoding='utf-8') as f:
        f.readline() # first line '['
        numReviews = 0
        while True:
            numReviews += 1
            line = f.readline().strip() # Get next line from file
            if line == ']': # end of file is reached ']'
                print(f'{data_name}: {numReviews} reviews loaded')
                break
            if line[-1] == ',':
                line = line[:-1]
            reviews.append(json.loads(line))
    return reviews

In [6]:
reviews_spas = load_by_line_json_yelp('yelp_beauty_spas')
print(reviews_spas[0])
print(reviews_spas[0].get('reviewerID'))

yelp_beauty_spas: 5580 reviews loaded
{'reviewerID': 'Xm8HXE1JHqscXe5BKf0GFQ', 'asin': 'WGNIYMeXPyoWav1APUq7jA', 'summary': 'summary', 'reviewText': "Good tattoo shop. Clean space, multiple artists to choose from and books of their work are available for you to look though and decide who's style most mirrors what you're looking for. I chose Jet to do a cover-up for me and he worked with me on the design and our ideas and communication flowed very well. He's a very personable guy, is friendly and keeps the conversation going while he's working on you, and he doesn't dick around (read: He starts to work and continues until the job is done). He's very professional and informative. Good customer service combines with talent at the craft.", 'overall': 4.0}
Xm8HXE1JHqscXe5BKf0GFQ


In [7]:
reviews_restaurants = load_by_line_json_yelp('yelp_restaurants')
print(reviews_restaurants[0])
print(reviews_restaurants[0].get('reviewerID'))

yelp_restaurants: 158431 reviews loaded
{'reviewerID': 'rLtl8ZkDX5vH5nAx9C3q5Q', 'asin': '9yKzy9PApeiPPOUJEtnvkg', 'summary': 'summary', 'reviewText': 'My wife took me here on my birthday for breakfast and it was excellent. The weather was perfect which made sitting outside overlooking their grounds an absolute pleasure. Our waitress was excellent and our food arrived quickly on the semi-busy Saturday morning. It looked like the place fills up pretty quickly so the earlier you get here the better.Do yourself a favor and get their Bloody Mary. It was phenomenal and simply the best I\'ve ever had. I\'m pretty sure they only use ingredients from their garden and blend them fresh when you order it. It was amazing.While EVERYTHING on the menu looks excellent, I had the white truffle scrambled eggs vegetable skillet and it was tasty and delicious. It came with 2 pieces of their griddled bread with was amazing and it absolutely made the meal complete. It was the best "toast" I\'ve ever had.An

### Task 1.3
Loading line by line* reviews on other domains like digital music from McAuley’s Amazon dataset2.

Una opción de leer linea por linea un json muy grande es utilizar la función *read_json* de pandas con el atributo 'lines' a True. Después se ha realizado una limpieza del dataframe (eliminado columnas que no estuviesen en el dataset yelp)

In [8]:
def read_by_line_json_amazon(data_name, data_path = 'amazon_dataset'):
    df = pd.read_json(f'{data_path}/{data_name}.json', lines=True)
    df.drop(inplace=True, columns=['verified', 'reviewTime', 'reviewerName', 'reviewText', 'unixReviewTime', 'style', 'image', 'vote'])
    return df.to_dict('records')

In [22]:
fashion_reviews = read_by_line_json_amazon('digital_music')
print(fashion_reviews[0])
print(fashion_reviews[0].get('reviewerID'))

{'overall': 5, 'reviewerID': 'A1ZCPG3D3HGRSS', 'asin': '0001388703', 'summary': 'Great worship cd'}
A1ZCPG3D3HGRSS


## Assignment 2: Aspect vocabularies

### Task 2.1
Loading (and printing on screen) the vocabulary of the aspects_hotels.csv
file, and directly using it to identify aspect references in the reviews. In particular, the aspects terms
could be mapped by exact matching with nouns appearing in the reviews. 

In [8]:
def load_aspects(data_name, data_path = 'aspects'):
    with open(f'{data_path}/{data_name}.csv', encoding='utf-8') as f:
        aspects = {}
        for line in f:
            key, synonymous = line.rstrip('\n').split(',')
            if key in aspects and synonymous not in aspects[key]:
                aspects[key].append(synonymous)
            else:
                aspects[key] = []
    return aspects

In [9]:
aspects_hotels = load_aspects('aspects_hotels')
aspects_hotels

{'amenities': ['amenities', 'services'],
 'atmosphere': ['atmospheres',
  'ambiance',
  'ambiances',
  'light',
  'lighting',
  'lights',
  'music'],
 'bar': ['bars', 'bartender', 'bartenders'],
 'bathrooms': ['bathrooms',
  'bath',
  'baths',
  'bathtub',
  'bathtubs',
  'shampoo',
  'shampoos',
  'shower',
  'showers',
  'towel',
  'towels',
  'tub',
  'tubs'],
 'bedrooms': ['bedrooms',
  'bed',
  'beds',
  'pillow',
  'pillows',
  'sheet',
  'sheets',
  'sleep',
  'suite',
  'suites'],
 'booking': ['book', 'reservation', 'reservations', 'reserve'],
 'breakfast': ['breakfasts',
  'morning',
  'mornings',
  'toast',
  'toasts',
  'moorning meal',
  'moorning menu'],
 'building': ['decor',
  'decoration',
  'decorations',
  'furniture',
  'furnitures',
  'garden',
  'gardens',
  'hall',
  'halls',
  'lobbies',
  'lobby',
  'lounge',
  'lounges',
  'patio',
  'patios',
  'salon',
  'salons',
  'spot',
  'spots'],
 'checking': ['check-in',
  'check in',
  'check ins',
  'check out',
  'c

### Task 2.2 

Generating or extending the lists of terms of each aspect with synonyms extracted from WordNet

In [10]:
def extend_aspects(aspects):
    for key in aspects:
        synsets = wn.synsets(key)
        for synset in synsets:
            lemmas = synset.lemma_names()
            aspects[key] = list(set(aspects[key]  + lemmas))
    return aspects

In [11]:
aspects_hotels = extend_aspects(aspects_hotels)
aspects_hotels

{'amenities': ['amenities',
  'conveniences',
  'amenity',
  'services',
  'comforts',
  'creature_comforts',
  'agreeableness'],
 'atmosphere': ['light',
  'lighting',
  'atmosphere',
  'ambience',
  'ambiance',
  'lights',
  'standard_atmosphere',
  'atmospheric_state',
  'music',
  'ambiances',
  'atmospheres',
  'atm',
  'aura',
  'air',
  'standard_pressure'],
 'bar': ['saloon',
  'block_off',
  'exclude',
  'Browning_automatic_rifle',
  'taproom',
  'barricade',
  'legal_profession',
  'stop',
  'prevention',
  'block_up',
  'bars',
  'block',
  'relegate',
  'blockade',
  'BAR',
  'banish',
  'barroom',
  'bartenders',
  'streak',
  'ginmill',
  'bartender',
  'debar',
  'cake',
  'bar',
  'stripe',
  'measure',
  'legal_community'],
 'bathrooms': ['towels',
  'lavatory',
  'bathtubs',
  'shampoo',
  'lav',
  'towel',
  'john',
  'tubs',
  'bath',
  'can',
  'baths',
  'bathroom',
  'bathtub',
  'bathrooms',
  'shampoos',
  'toilet',
  'privy',
  'shower',
  'tub',
  'showers'],

### Task 2.3 
Managing vocabularies for additional Yelp or Amazon domains.

In [12]:
aspects_spas = load_aspects('aspects_spas')
aspects_spas = extend_aspects(aspects_spas)
aspects_spas

{'amenities': ['amenities',
  'conveniences',
  'amenity',
  'services',
  'comforts',
  'creature_comforts',
  'agreeableness'],
 'atmosphere': ['light',
  'lighting',
  'atmosphere',
  'ambience',
  'ambiance',
  'lights',
  'standard_atmosphere',
  'atmospheric_state',
  'music',
  'ambiances',
  'atmospheres',
  'atm',
  'ambiences',
  'aura',
  'air',
  'standard_pressure'],
 'bar': ['saloon',
  'block_off',
  'exclude',
  'Browning_automatic_rifle',
  'taproom',
  'barricade',
  'legal_profession',
  'stop',
  'prevention',
  'block_up',
  'bars',
  'block',
  'relegate',
  'blockade',
  'BAR',
  'banish',
  'barroom',
  'bartenders',
  'streak',
  'ginmill',
  'bartender',
  'debar',
  'cake',
  'bar',
  'stripe',
  'measure',
  'legal_community'],
 'bathrooms': ['towels',
  'lavatory',
  'bathtubs',
  'shampoo',
  'lav',
  'towel',
  'john',
  'tubs',
  'bath',
  'can',
  'baths',
  'bathroom',
  'bathtub',
  'bathrooms',
  'shampoos',
  'toilet',
  'privy',
  'shower',
  'tub'

In [13]:
aspects_restaurants = load_aspects('aspects_restaurants')
aspects_restaurants = extend_aspects(aspects_restaurants)
aspects_restaurants

{'appetizers': ['entrees',
  'starters',
  'starter',
  'appetizer',
  'appetizers',
  'appetiser',
  'entree'],
 'asian': ['noodles',
  'sushi',
  'noodle',
  'sushies',
  'Asiatic',
  'curries',
  'Asian',
  'curry'],
 'atmosphere': ['light',
  'lighting',
  'atmosphere',
  'ambience',
  'ambiance',
  'lights',
  'standard_atmosphere',
  'atmospheric_state',
  'music',
  'ambiances',
  'atmospheres',
  'atm',
  'aura',
  'air',
  'standard_pressure'],
 'bar': ['saloon',
  'block_off',
  'exclude',
  'Browning_automatic_rifle',
  'taproom',
  'barricade',
  'legal_profession',
  'stop',
  'prevention',
  'block_up',
  'bars',
  'block',
  'relegate',
  'blockade',
  'BAR',
  'banish',
  'barroom',
  'bartenders',
  'streak',
  'ginmill',
  'bartender',
  'debar',
  'cake',
  'bar',
  'stripe',
  'measure',
  'legal_community'],
 'booking': ['reserve',
  'hold',
  'reservations',
  'book',
  'engagement',
  'booking',
  'reservation'],
 'bread': ['pelf',
  'clams',
  'lolly',
  'breads

### Task 2.4
Identifying hidden/implicit aspect references in reviews. For instance, the example review of page 1 has references to the hotel’s location and transportation aspects, since there is “not much around the area” and "going by car to the hotel is recommendable".


For this task, we are going to considerer the hyponym of words. For example, that 'area' is a hyponym of 'location'.

## Assignment 3: Opinion Lexicon

### Task 3.1

Loading Liu’s opinion lexicon composed of positive and negative words, accessible as an NLKT corpus, and exploiting it to assign the polarity values to aspect opinions in assignment 4. Instead of this lexicon, you are allowed to use others, such as SentiWordNet.

In [29]:
import nltk
#nltk.download('opinion_lexicon')
from nltk.corpus import opinion_lexicon

negativeWords = opinion_lexicon.negative()
positiveWords = opinion_lexicon.positive()

['a+', 'abound', 'abounds', 'abundance', 'abundant', ...]

In [72]:
from nltk.corpus import sentiwordnet as swn
#nltk.download('sentiwordnet')

# lexicon values between -1,1
lexicon = {}
for negativeWord in negativeWords:
    for senti_word in swn.senti_synsets(negativeWord):
        lexicon[senti_word] = float(- senti_word.neg_score())
for positiveWord in positiveWords:
    for senti_word in swn.senti_synsets(positiveWord):
        lexicon[senti_word] = float(senti_word.pos_score())

In [63]:
import spacy
#!python -m spacy download en_core_web_lg
file = nltk.data.load("vader_lexicon/vader_lexicon.txt")
nlp = spacy.load("en_core_web_lg")

In [64]:
# lexicon2 between -3, 3
lexicon2 = {}
# $:	-1.5	0.80623	[-1, -1, -1, -1, -3, -1, -3, -1, -2, -1]
for l in file.split("\n"):
    word, polarity = l.strip().split("\t")[0:2]
    lexicon2[word] = float(polarity)

In [66]:
doc = nlp("I do not think the hotel staff was friendly")
displacy.render(doc, style="dep")

### Task 3.2

Considering modifiers to adjust the polarity values of the aspect opinions in Assignment 4. The modifiers to use could be those provided with the NLTK Sentiment Analyzer (see Appendix G) and/or those given in modifiers.csv.

In [67]:
modifiers = pd.read_csv("modifiers/modifiers.csv")
modifiers

,above,2
0,absolutely,2.0
1,abundantly,2.0
2,acutely,2.0
3,amazingly,2.0
4,amply,2.0
...,...,...
294,violently,-1.0
295,whimsically,-1.0
296,wickedly,-1.0
297,wretchedly,-1.0


In [68]:
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.sentiment.vader import VaderConstants

constants = VaderConstants()
print(constants.BOOSTER_DICT)

{'absolutely': 0.293, 'amazingly': 0.293, 'awfully': 0.293, 'completely': 0.293, 'considerably': 0.293, 'decidedly': 0.293, 'deeply': 0.293, 'effing': 0.293, 'enormously': 0.293, 'entirely': 0.293, 'especially': 0.293, 'exceptionally': 0.293, 'extremely': 0.293, 'fabulously': 0.293, 'flipping': 0.293, 'flippin': 0.293, 'fricking': 0.293, 'frickin': 0.293, 'frigging': 0.293, 'friggin': 0.293, 'fully': 0.293, 'fucking': 0.293, 'greatly': 0.293, 'hella': 0.293, 'highly': 0.293, 'hugely': 0.293, 'incredibly': 0.293, 'intensely': 0.293, 'majorly': 0.293, 'more': 0.293, 'most': 0.293, 'particularly': 0.293, 'purely': 0.293, 'quite': 0.293, 'really': 0.293, 'remarkably': 0.293, 'so': 0.293, 'substantially': 0.293, 'thoroughly': 0.293, 'totally': 0.293, 'tremendously': 0.293, 'uber': 0.293, 'unbelievably': 0.293, 'unusually': 0.293, 'utterly': 0.293, 'very': 0.293, 'almost': -0.293, 'barely': -0.293, 'hardly': -0.293, 'just enough': -0.293, 'kind of': -0.293, 'kinda': -0.293, 'kindof': -0.293,

In [99]:
#nltk.download('vader_lexicon')

analyzer = SentimentIntensityAnalyzer()
s = "I'm really interested in music"
print(s, " => ", analyzer.polarity_scores(s))

I'm really interested in music  =>  {'neg': 0.0, 'neu': 0.572, 'pos': 0.428, 'compound': 0.4576}


## Assignment 4: Aspect opinions

Once the aspect vocabulary and opinion lexicons are loaded, the opinions about aspects have to be extracted
from the reviews. For this purpose, POS tagging, constituency and dependency parsing could be used.
- POS tagging would allow identifying the adjectives in the sentences.
- Constituency and dependency parsing would allow extracting the relations between nouns and adjectives and adverbs.

#### Task 4.1 -: 
Extracting the [aspect, aspect term, opinion word, polarity] tuples from the input reviews

In [140]:
def get_aspect(word, aspects):
    for key in aspects:
        if word in aspects[key]:
            return key

get_aspect('place', aspects_hotels)

In [241]:
import pandas as pd

review = reviews_hotels[0].get("reviewText")

from nltk.parse.corenlp import CoreNLPDependencyParser
dependency_parser = CoreNLPDependencyParser()


sentences = nltk.sent_tokenize(review)

data = pd.DataFrame(columns = ['polarity', 'aspect', 'adjetive', 'word'])
for s in sentences:
    print(s)
    
    result, = dependency_parser.raw_parse(s)
    
    dict_inf = {}
    for head, relation, dependent in result.triples():
        word1, pos1 = head
        word2, pos2 = dependent        
        
        if pos1.startswith('NN') and pos2.startswith('JJ') and relation == 'amod':
            
            if pos1 == 'NN':
                aspect = get_aspect(word1, aspects_hotels)
            if 'adjetive' in dict_inf:
                if aspect != None:
                    dict_inf['aspect'] = aspect.upper()
                    
                    polarity_scores = analyzer.polarity_scores(dict_inf['adjetive'] +' '+ dict_inf["word"])
                    if polarity_scores['pos'] > polarity_scores['neg']:
                        polarity = 1.0
                    else:
                        polarity = -1.0

                    dict_inf['polarity'] = polarity
        
                    
                    data = data.append(dict_inf, ignore_index=True)
                    aspect = get_aspect(aspect, aspects_hotels)
                    dict_inf = {}
                
            dict_inf['adjetive'] = word2.lower()
            dict_inf['word'] = word1.lower()
        elif pos1.startswith('NN') and pos2.startswith('NN') and relation == 'compound':
            
            if 'word' in dict_inf and word1 in dict_inf['word']:
                word_list =  dict_inf['word'].split()
                word_list.insert(-1, word2)
                dict_inf['word'] = ' '.join(word_list).lower()
            if pos1 == 'NNS' and pos2=='NN':
                aspect = get_aspect(word2, aspects_hotels)
        elif pos1.startswith('JJ') and pos2.startswith('RB') and relation == 'advmod':
            if 'adjetive' in dict_inf and word1 in dict_inf['adjetive']:
                adj_list =  dict_inf['adjetive'].split()
                adj_list.insert(-1, word2)
                dict_inf['adjetive'] = ' '.join(adj_list).lower()
            
    if aspect != None:
        dict_inf['aspect'] = aspect.upper()
        
        polarity_scores = analyzer.polarity_scores(dict_inf['adjetive'] +' '+ dict_inf["word"])
        if polarity_scores['pos'] > polarity_scores['neg']:
            polarity = 1.0
        else:
            polarity = -1.0
        
        dict_inf['polarity'] = polarity
        data = data.append(dict_inf, ignore_index=True)
        aspect = None
data

Great hotel in Central Phoenix for a stay-cation, but not necessarily a place to stay out of town and without a car.
Not much around the area, and unless you're familiar with downtown, I would rather have a guest stay in Old Town Scottsdale, etc.
BUT if you do stay here, it's awesome.
Great boutique rooms.
Awesome pool that's happening in the summer.
A GREAT rooftop patio bar, and a very very busy lobby with Gallo Blanco attached.
A great place to stay, but have a car!


,polarity,aspect,adjetive,word
0,1.0,SHOPPING,great,boutique rooms
1,1.0,POOL,awesome,pool
2,1.0,BUILDING,great,rooftop patio bar
3,1.0,BUILDING,very very busy,lobby


In [77]:
def pos_tagging(text):
    sentences = nltk.sent_tokenize(text)
    sentences = [nltk.word_tokenize(s) for s in sentences]
    sentences = [nltk.pos_tag(s) for s in sentences]
    return sentences

postagged_sentences = pos_tagging(reviews_hotels[0].get("reviewText"))
print(postagged_sentences[3])

[('Great', 'NNP'), ('boutique', 'NN'), ('rooms', 'NNS'), ('.', '.')]
